https://github.com/jacobgil/keras-dcgan/blob/master/dcgan.py

In [3]:
import sys
sys.path.append('../notebooks')
import utils
import os
import numpy as np
import glob
import cv2
import utils
import pylab as pl
import shutil
import matplotlib.cm as cm
from sklearn.cross_validation import train_test_split
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/ipykernel/.*')
%matplotlib inline

In [9]:
_d = np.load('../data/data_128_train.npz')
X_train = _d['X'][:,None,:,:]
Y_train = _d['Y']

In [24]:
# set X_train to be in [-1, 1]
X_train = (X_train - 0.5) * 2

In [25]:
print X_train.shape
print Y_train.shape

(5635, 1, 128, 128)
(5635, 128, 128)


In [26]:
# channels first in keras
input_shape = X_train[0].shape
print input_shape

(1, 128, 128)


In [63]:
os.environ['KERAS_BACKEND'] = 'theano'
# TODO: Use float16 ?
os.environ['THEANO_FLAGS'] = 'mode=FAST_RUN,device=gpu0,floatX=float16'
import keras
from keras.layers import Input, Dense, Dropout, Activation, Flatten, merge, RepeatVector, Permute, Reshape
from keras.layers import Convolution2D, MaxPooling2D, UpSampling2D, MaxoutDense, BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Model, Sequential
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import print_summary
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint

In [28]:
import theano
print theano.config.device

gpu0


In [87]:
NOISE_SIZE = 20

def get_batches(batch_size):
    for i in range(X_train.shape[0]/batch_size):
        yield i, X_train[i*batch_size : (i + 1) * batch_size]

# -------------- Generator
gen_input = x = Input(shape=(NOISE_SIZE,), name='gen_input')
x = Dense(64*4*4)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Reshape((64, 4, 4))(x)
x = UpSampling2D(size=(2, 2))(x)
x = Convolution2D(32, 5, 5, border_mode='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D(size=(2, 2))(x)
x = Convolution2D(32, 5, 5, border_mode='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D(size=(2, 2))(x)
x = Convolution2D(32, 5, 5, border_mode='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D(size=(2, 2))(x)
x = Convolution2D(32, 5, 5, border_mode='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D(size=(2, 2))(x)
x = Convolution2D(1, 5, 5, border_mode='same')(x)
gen_out = x = Activation('tanh')(x)

generator = Model(
    input=gen_input,
    output=gen_out
)

# -------------- Discriminator
disc_input = x = Input(shape=(1, 128, 128), name='disc_input')
x = Convolution2D(32, 5, 5, subsample=(2, 2), border_mode='same')(x)
x = LeakyReLU(0.2)(x)
x = BatchNormalization()(x)
x = Convolution2D(32, 5, 5, subsample=(2, 2), border_mode='same')(x)
x = LeakyReLU(0.2)(x)
x = BatchNormalization()(x)
x = Convolution2D(32, 5, 5, subsample=(2, 2), border_mode='same')(x)
x = LeakyReLU(0.2)(x)
x = BatchNormalization()(x)
x = Convolution2D(32, 5, 5, subsample=(2, 2), border_mode='same')(x)
x = LeakyReLU(0.2)(x)
x = BatchNormalization()(x)
x = Convolution2D(32, 5, 5, subsample=(2, 2), border_mode='same')(x)
x = LeakyReLU(0.2)(x)
x = BatchNormalization()(x)
x = Flatten()(x)
disc_out = Dense(1, activation='sigmoid')(x)
discriminator = Model(
    input=disc_input,
    output=disc_out
)

# -------------- Both chained
disc_on_gen_input = Input(shape=(NOISE_SIZE,), name='disc_on_gen_input')
for l in discriminator.layers:
    l.trainable = False
discriminator_on_generator = Model(
    input=disc_on_gen_input,
    output=discriminator(generator(disc_on_gen_input))
)

adam=Adam(lr=0.0002, beta_1=0.5, beta_2=0.999, epsilon=1e-08)
generator.compile(loss='binary_crossentropy', optimizer=adam)

discriminator_on_generator.compile(loss='binary_crossentropy', optimizer=adam)

for l in discriminator.layers:
    l.trainable = True
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

BATCH_SIZE=64

for epoch in range(5):
    print "Epoch is", epoch
    print "Number of batches", X_train.shape[0] / BATCH_SIZE
    for index, image_batch in get_batches(batch_size=BATCH_SIZE):
        noise = np.zeros((BATCH_SIZE, NOISE_SIZE))
        for i in range(BATCH_SIZE):
            noise[i, : ] = np.random.uniform(-1, 1, NOISE_SIZE)

        print 'Generating images..'
        generated_images = generator.predict(noise)
        print 'Generated..'
        for i, img in enumerate(generated_images):
            rolled = np.rollaxis(img, 0, 3)
            cv2.imwrite(str(i) + ".jpg", np.uint8(255 * 0.5 * (rolled + 1.0)))

        X = np.concatenate((image_batch, generated_images))

        y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
        print "Batch", index, "Training discriminator.."
        d_loss = discriminator.train_on_batch(X, y)

        for j in range(1):
            noise = np.zeros((BATCH_SIZE, NOISE_SIZE))
            for i in range(BATCH_SIZE):
                noise[i, : ] = np.random.uniform(-1, 1, NOISE_SIZE)

            print "Training generator.."
            g_loss = discriminator_on_generator.train_on_batch(noise, [1] * BATCH_SIZE)
            print "Generator loss", g_loss, "Discriminator loss", d_loss, "Total:", g_loss[0] + d_loss[0]

        if index % 10 == 9:
            print 'Saving weights..'
            generator.save_weights('generator', True)
            discriminator.save_weights('discriminator', True)

Epoch is 0
Number of batches 88
Generating images..
Generated..
Batch 0 Training discriminator..
Training generator..


MissingInputError: ("An input of the graph, used to compute dot(gen_input, HostFromGpu.0), was not provided and not given a value.Use the Theano flag exception_verbosity='high',for more information on this error.", gen_input)

In [78]:
#g_loss = discriminator_on_generator.fit(noise, [1] * BATCH_SIZE)
discriminator_on_generator.get_input_shape_at(0)

(None, 20)

In [ ]:
g_loss = discriminator_on_generator.fit

In [74]:
print_summary(generator.layers)
print_summary(discriminator.layers)
print_summary(discriminator_on_generator.layers)

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
gen_input (InputLayer)             (None, 20)          0                                            
____________________________________________________________________________________________________
dense_31 (Dense)                   (None, 1024)        21504       gen_input[0][0]                  
____________________________________________________________________________________________________
batchnormalization_152 (BatchNormal(None, 1024)        2048        dense_31[0][0]                   
____________________________________________________________________________________________________
activation_96 (Activation)         (None, 1024)        0           batchnormalization_152[0][0]     
___________________________________________________________________________________________